# Natural Language Processing

So what is natural language processing? What kinds of data falls into the "natural language" bucket? What types of data are similar that you've already seen? What topics have we already discussed in the curriculum that lead up to this section?

- 


## Processing Text Data

Using the 'Spooky Authors' dataset: https://www.kaggle.com/c/spooky-author-identification/overview

Goal: determine whether a chunk of text was written by Edgar Allen Poe, HP Lovecraft or Mary Shelley (multi-class classification problem)

In [ ]:
# Wall of imports
import pandas as pd
import string
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, plot_confusion_matrix

First let's grab the dataset and look at a few aspects of this dataset (shape, some examples, etc). We'll be using just the train csv for this, for ease of use!

In [ ]:
# Grab the train set from the competition 


In [ ]:
# Exploring an example of one of the texts


In [ ]:
# Encoding our target from author initials to numbers


In [ ]:
# Checking that change


In [ ]:
# Grabbing our inputs and target
X = None
y = None

# Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
# Grabbing a list of stopwords from NLTK, imported above
# We're also using the string library add punctuation to our list
stopwords_list = stopwords.words('english') + list(string.punctuation)

What is the point of a list of stopwords? How/why will we use this list?

- 


### "Bag of Words" - Count Vectorizer

Useful link to the 'User Guide' part of the documentation on this: https://scikit-learn.org/stable/modules/feature_extraction.html#common-vectorizer-usage

In [ ]:
# Intstantiating our vectorizer


# Training on the train set, then transforming the train set

# Transforming the test set


In [ ]:
# Instantiating a classifier to use on this text - Multinomial Naive Bayes


# Fitting the classifier


# Getting our predictions for the train and test sets



In [ ]:
# Let's see how we did!


Discuss! How did we do? What could we change?

- 


We're about to try this on a few different vectorizers, so let's make that easier!

Here, I'm writing a function where we can provide an instantiated vectorizer, an instantiated classifer, and all of our train and test data, and the function will spit out the accuracy score and confusion matrix just like above:

In [ ]:
def classify_vectorized_text(vectorizer, classifier, Xtrain, Xtest, ytrain, ytest):
    '''
    Fit and transform text data using the provided vectorizer, then fit and 
    predict with the provided classifier, in order to see the resulting
    accuracy score and confusion matrix
    For the Xtrain, Xtest, ytrain, ytest, expect the output of an
    sklearn train/test split
    -
    Inputs:
    vectorizer: an instantiated sklearn vectorizer
    classifier: an instantiated sklearn classifier
    X_train: training input data
    X_test: testing input data
    y_train: training true result
    y_test: testing true result
    -
    Outputs: 
    train_preds: predicted results for the train set
    test_preds: predicted results for the test set
    '''

    Xtrain_transformed = vectorizer.fit_transform(Xtrain)
    Xtest_transformed = vectorizer.transform(Xtest)

    classifier.fit(Xtrain_transformed, ytrain)

    train_preds = classifier.predict(Xtrain_transformed)
    test_preds = classifier.predict(Xtest_transformed)

    print(accuracy_score(ytest, test_preds))
    plot_confusion_matrix(classifier, Xtest_transformed, ytest,
                          values_format=".4g")  # to make numbers readable
    plt.show()

    return(train_preds, test_preds)

Let's add in something that was missing from our first Count Vectorizer:

Link to the documentation: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

In [ ]:
# Instantiating a count vectorizer that removes stop words


Now let's create a new classifier and compare the results, using our previously-defined function!

In [ ]:
# Trying the same classifier, but now with stopwords removed


Compare/discuss: 

- 


### TF-IDF: Term-Frequency - Inverse Document-Frequency

Woah, that's a term and a half. What even is it?

From [the documentation](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html):

> "The goal of using tf-idf instead of the raw frequencies of occurrence of a token in a given document is to scale down the impact of tokens that occur very frequently in a given corpus and that are hence empirically less informative than features that occur in a small fraction of the training corpus."

Basically, it's a statistic that hopefully reflects how important a word is in the document. By looking at the overall frequency you find how common a word is across the whole corpus, compared to the document frequency that shows how common a word is within the document in question. If a word appears often in our document, but relatively rarely in the corpus, it probably captures an important word in that specific document!

In this example, the training corpus is every sentence in the `text` column in our train set, and the document is the individual sentence that we're trying to classify (per row).

Reference: http://www.tfidf.com/

We'll be using Sklearn's [`TfidfVectorizer`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html), which is 'equivalent to CountVectorizer followed by TfidfTransformer'

In [ ]:
# Instantiating the TF-IDF vectorizer
tfidf = TfidfVectorizer(stop_words=stopwords_list, 
                        max_df = .95, # removes words that appear in more than 95% of docs
                        min_df = 2, # removes words that appear 2 or fewer times
                        use_idf=True)

# Training on the train set, then transforming the train set

# Transforming the test set


In [ ]:
# Here, number of rows is the length of our train set


In [ ]:
# Here, number of rows is the length of our test set


We can actually explore what our vectorizer is grabbing from the test data.

In [ ]:
# Let's look at a doc in our test set
X_test.iloc[15]

In [ ]:
# Creating a df of tf-idf values, where each column is a word in the vocabulary
tfidf_test_df = pd.DataFrame(tfidf_test.toarray(), columns=tfidf.get_feature_names())

In [ ]:
# Grabbing that row once it's been vectorized
test_doc = tfidf_test_df.iloc[15]

print(test_doc.loc[test_doc > 0].sort_values(ascending=False)) # Showing values > 0

This tells you that for the 16th document in our test set, the word 'wake' has the highest TF-IDF value.

What does this tell you about the word "wake" in the this document of our test set?

- 


In [ ]:
# Let's compare to our earlier count vectorizer for this document
count_test_df = pd.DataFrame(count_stop_test.toarray(), columns=count_stop.get_feature_names())

In [ ]:
count_test_doc = count_test_df.iloc[15]

print(count_test_doc.loc[count_test_doc > 0].sort_values(ascending=False)) # Showing values > 0

Easy to discuss here why using a TF-IDF Vectorizer might have some added benefits compared to a straight Count Vectorizer.

In [ ]:
# Now, back to classifying: 
# Using our function to compare the results...



Compare/discuss:

- 


In [ ]:
# We can also use our function to try different classifiers



Compare/discuss: 

- 


## Additional Resources

- Sklearn's [Working with Text Data Tutorial](https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html)

What else can we do with natural language data beyond text classification? 

- [This blog post](https://blog.aureusanalytics.com/blog/5-natural-language-processing-techniques-for-extracting-information) by Aureus Analytics provides an overview of other machine learning techniques used to extract meaning from text: Named Entity Recognition, Sentiment Analysis, Text Summarization, Aspect Mining and Topic Modeling

### Neural Network Vectorizer Resources:

Want to go beyond Count Vectorizers or TF-IDF to embed words for machine learning? Check out Word2Vec - a way of vectorizing text that tries to capture the relationships between words. See the image below, from [this paper](https://arxiv.org/pdf/1310.4546.pdf) from Google developers, that introduced a Skip-gram neural network model that's been utilized by Word2Vec (which is a tool you can use to implement this model). You'll note that the distance between each country and it's capital city is about the same - that distance actually has meaning, and thus you can imagine that the difference between `cat` and `kitten` would be the same as the difference between `dog` and `puppy`. Et cetera!

![screenshot from a paper on the Skip-gram model from devleopers at Google, https://arxiv.org/pdf/1310.4546.pdf](images/Fig2-DsitributedRepresentationsOfWordsAndPhrasesAndTheirCompositionality.png)

- [Pathmind's A.I. Wiki - A Beginner's Guide to Word2Vec](https://wiki.pathmind.com/word2vec)
- [Chris McCormick's Word2Vec Tutorial](http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/)